In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def cal_snp_per_gene(df_gene, df_snp):
    '''
    Compute SNP positions in each gene
    input:
        df_gene - gene DataFrame
        df_snp - SNP DataFrame
    ouput:
        dic_gene_snp: a dictionary from gene to all its SNPs; key: gene name, value: a list of its SNP positions    
    '''
      
    n_gene, n_snp = len(df_gene), len(df_snp)
    
    dic_gene_snp = { df_gene['gene'][i] : [] for i in range(n_gene)}
    for i in range(n_snp):
        chrom, pos = df_snp['chromosome'][i], df_snp['position'][i]
        mask = (pos > df_gene['position_left']) & (pos < df_gene['position_right']) \
                & (df_gene['chromosome']==chrom)
        
        if mask.sum()==1:
            gene_name = df_gene['gene'][mask].iloc[0]
            dic_gene_snp[ gene_name ].append(pos)
            
        if (i % (int(len(df_snp) / 10)) == 0):
            print(i / (int(len(df_snp) / 100))), "% complete")
    return dic_gene_snp   

In [ ]:
if __name__=='__main__':
    
    '''data is from https://github.com/mckenaliphamr/GoGreen/tree/master/sample_data'''
    #path_gene_data = 'data\\genes_w_features_downsample.csv'
    path_gene_data = 'C:/Users/15099/Documents/School/MSU_BMS/Spring_2020/condensed_subset_Davis.csv'
    path_snp_data = 'C:/Users/15099/Documents/School/MSU_BMS/Spring_2020/condensed_SNP_subset_Davis.csv'
    df_gene = pd.read_csv(path_gene_data, sep='\t')
    df_snp = pd.read_csv(path_snp_data, sep='\t')
    dic_gene_snp = cal_snp_per_gene(df_gene, df_snp)

In [ ]:
#initialize values of density graph as 0
density = np.zeros(1000)

#loop through each gene and quantify location of SNPs
for i in range(len(df_gene)):
    
    # calculate gene length based off of the difference in position
    gene_length = df_gene['position_right'][i] - df_gene['position_left'][i]
    
    # investigates SNPs in each gene individually
    SNPs_each_gene = dic_gene_snp[df_gene['gene'].values[i]]
    
    # normalizes position of SNPs to gene length and multiplies by 1000 for positioning
    SNPs_each_gene = ((SNPs_each_gene - df_gene['position_left'].values[i]) / gene_length) * 1000
    
    # loop through each g
    for j in range(0, len(SNPs_each_gene)):
        if (df_gene['Direction'][i]=='-'):
            SNPs_each_gene[j] = 1000 - int(SNPs_each_gene[j])
        else:
            SNPs_each_gene[j] = int(SNPs_each_gene[j])
    for j in range(0, len(density)):
        if j in SNPs_each_gene:
            density[j] = density[j] + 1

In [ ]:
abundancy_normalization = max(density)
density_normalized = (density / abundancy_normalization) * 100
fig = plt.figure(figsize=(12,7))

plt.plot(density_normalized, c='seagreen', linewidth=3.0)
plt.xlabel('Relative Position within the Transcript', fontsize=16)
plt.ylabel('Comparative SNP Accumulation', fontsize=16)
plt.title('SNP Density', fontsize=20)
plt.xlim(-4, 1003)

In [ ]:
abundancy_normalization = len(df_gene)
density_relative = (density / abundancy_normalization) * 100
fig = plt.figure(figsize=(12,7))

plt.plot(density_relative, c='seagreen', linewidth=3.0)
plt.xlabel('Relative Position within the Transcript', fontsize=16)
plt.ylabel('Percent of Transcripts with SNP', fontsize=16)
plt.title('SNP Density', fontsize=20)
plt.xlim(-4, 1003)
plt.ylim(0, 5)